In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim
from torchvision import transforms
import torch.nn.functional as F
torch.manual_seed(22)

In [2]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [3]:
# file path
train_imgs_file = "data/kmnist-npz/kmnist-train-imgs.npz"
train_labels_file = "data/kmnist-npz/kmnist-train-labels.npz"

val_imgs_file = "data/kmnist-npz/kmnist-val-imgs.npz"
val_labels_file = "data/kmnist-npz/kmnist-val-labels.npz"

test_imgs_file = "data/kmnist-npz/kmnist-test-imgs.npz"
test_labels_file = "data/kmnist-npz/kmnist-test-labels.npz"

# Parameters
batchsize = 64 #64
epochs = 14
lr = 0.001  #smaller -> more likely to overfit
betas = (0.9, 0.999)

In [4]:
classes = ( 'お','き','す','つ','な','は','ま','や','れ','を')

In [5]:
# define transfrom for training set, composes several transforms together
transform = transforms.Compose([
    transforms.ToPILImage(),
    #randomly flip the image left and right
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.ToTensor(),
    # normalize the data to the range between (-1, 1)
    transforms.Normalize(mean=0.5, std=0.5),
    # zero-pad 4 pixels
    transforms.Pad([4, 4]),
    #randomly crop image to 28x28
    transforms.RandomCrop(28),
    ])

In [6]:
# # define transfrom for validation set & test set
transform2 = transforms.Compose([
    transforms.ToTensor(),
    # normalize the data to the range between (-1, 1)
    transforms.Normalize(mean=0.5, std=0.5),
    ])

In [7]:
# Dataset class
class MyDataset(Dataset):
    def __init__(self, imgs_file, labels_file, transform=None):
        self.imgs = np.load(imgs_file)['arr_0']
        self.img_labels = np.load(labels_file)['arr_0']
        self.img_dir = imgs_file
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.imgs[idx]
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [8]:
# Create instances of dataset and load training, validation, test sets
training_set = MyDataset(train_imgs_file, train_labels_file, transform)
train_loader = DataLoader(training_set, batch_size = batchsize, shuffle = True)

val_set = MyDataset(val_imgs_file, val_labels_file, transform2)
val_loader = DataLoader(val_set, batch_size = batchsize, shuffle = True)

test_set = MyDataset(test_imgs_file, test_labels_file, transform2)
test_loader = DataLoader(test_set, batch_size = batchsize, shuffle = True)

In [9]:
f1 = 64
f2 = 128

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # 5×5 Convolutional Layer with 32 filters, stride 1 and padding 2.
        self.conv1 = nn.Conv2d(1, 32, 5, stride = 1,padding =  2) #input channels, output channels: # of filters, kernel_size
        # 2×2 Max Pooling Layer with a stride of 2.
        self.max_pooling = nn.MaxPool2d(2, stride = 2)
        #3×3 Convolutional Layer with 64 filters, stride 1 and padding 1.
        self.conv2 = nn.Conv2d(f1, f2, 3, stride = 1,padding =  1)
        #Fully-connected layer with 1024 output units.
        self.fc1 = nn.Linear(f2*7*7, 1024) #output channels*image size
        # Second fully connected layer that outputs our 10 labels
        self.fc2 = nn.Linear(1024, 10)
        self.dropout = nn.Dropout(p = 0.2)
        self.batchnorm = nn.BatchNorm2d(f2)
        self.batchnorm32 = nn.BatchNorm2d(f1)
        self.conv3 = nn.Conv2d(f2, f2, 3, stride = 1,padding =  1)
        self.conv4 = nn.Conv2d(32, f1, 3, stride = 1,padding =  1)


    def forward(self, x):
        x = self.conv1(x)
        x = self.conv4(x)
        x = self.batchnorm32(x)
        x = F.relu(x)
        x = self.max_pooling(x)
        x= self.conv2(x)
       #  x = self.batchnorm(x)
        x = self.conv3(x)
        x = self.batchnorm(x)
        x = F.relu(x)
        x = self.max_pooling(x)
        x = torch.flatten(x, 1) #flatten all except batch
        #x = x.view(x.size(0), -1)
        x= F.relu(self.fc1(x))
       # x = self.dropout(x)
        x = self.fc2(x)
        return x#F.log_softmax(x, dim=0) 

In [10]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        # 5×5 Convolutional Layer with 32 filters, stride 1 and padding 2.
        self.conv1 = nn.Conv2d(1, 32, 5, stride = 1,padding =  2) #input channels, output channels: # of filters, kernel_size
        # 2×2 Max Pooling Layer with a stride of 2.
        self.max_pooling = nn.MaxPool2d(2, stride = 2)
        #3×3 Convolutional Layer with 64 filters, stride 1 and padding 1.
        self.conv2 = nn.Conv2d(32, 64, 3, stride = 1,padding =  1)
        #Fully-connected layer with 1024 output units.
        self.fc1 = nn.Linear(64*7*7, 1024) #output channels*image size
        # Second fully connected layer that outputs our 10 labels
        self.fc2 = nn.Linear(1024, 10)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pooling(x)
        x= F.relu(self.conv2(x))
        x = self.max_pooling(x)
        x = torch.flatten(x, 1) #flatten all except batch
        #x = x.view(x.size(0), -1)
        x= F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [61]:
# Create a instance of Model
model = Model()

In [62]:
train_features, train_labels = next(iter(train_loader))
output = model(train_features)

In [63]:
train_features.size()

torch.Size([64, 1, 28, 28])

In [64]:
_,predicted1 = torch.max(output.data, 1)

In [65]:
torch.max(output.data, 1)

torch.return_types.max(
values=tensor([-4.0486, -4.0971, -4.0344, -4.0348, -4.0655, -4.0902, -4.0567, -4.0746,
        -4.1026, -4.0965, -4.1252, -4.1157, -4.0640, -4.0466, -4.0253, -4.0702,
        -4.1113, -4.0086, -4.1035, -4.0846, -4.1179, -4.0786, -4.0855, -4.0877,
        -4.0754, -4.0897, -4.0770, -4.1156, -4.0987, -4.0990, -4.0846, -4.0591,
        -4.0843, -4.0714, -4.1043, -4.0841, -4.0740, -4.0472, -4.1000, -4.1300,
        -4.0722, -4.0738, -4.0878, -4.0576, -4.0558, -4.0232, -4.1070, -4.1022,
        -4.0636, -4.0758, -4.0520, -4.0833, -4.0913, -4.0698, -4.0569, -4.0739,
        -4.1121, -4.0733, -4.0521, -4.0992, -4.0693, -4.1051, -4.0607, -4.0767]),
indices=tensor([2, 9, 5, 2, 1, 1, 3, 6, 3, 3, 2, 6, 8, 8, 6, 5, 2, 8, 8, 7, 8, 4, 0, 8,
        8, 6, 3, 1, 0, 1, 1, 6, 0, 7, 2, 6, 0, 7, 4, 2, 0, 5, 8, 5, 4, 8, 4, 8,
        8, 0, 1, 9, 8, 1, 6, 9, 3, 3, 0, 7, 9, 1, 5, 6]))

In [55]:
predicted1

tensor([6, 6, 5, 5, 0, 1, 8, 1, 4, 4, 1, 4, 0, 4, 2, 1, 8, 0, 4, 8, 1, 8, 3, 5,
        5, 5, 3, 2, 7, 4, 5, 9, 3, 6, 8, 3, 5, 2, 9, 2, 4, 1, 5, 7, 4, 2, 8, 2,
        1, 9, 6, 0, 0, 6, 9, 3, 3, 7, 7, 3, 6, 1, 2, 2])

In [45]:
train_labels

tensor([8, 3, 1, 3, 2, 0, 2, 0, 0, 8, 6, 4, 3, 6, 0, 6, 2, 0, 6, 0, 3, 1, 9, 1,
        8, 6, 6, 7, 3, 7, 4, 3, 5, 2, 8, 1, 3, 0, 8, 7, 2, 7, 2, 3, 4, 5, 3, 5,
        9, 4, 6, 5, 1, 3, 6, 1, 8, 1, 5, 7, 4, 7, 3, 6], dtype=torch.uint8)

In [46]:
output.shape

torch.Size([64, 10])

In [47]:
output.view(output.size(0), -1).shape

torch.Size([64, 10])

In [59]:
output[0]

tensor([0.0152, 0.0147, 0.0152, 0.0158, 0.0164, 0.0157, 0.0167, 0.0161, 0.0166,
        0.0150], grad_fn=<SelectBackward0>)

In [144]:
cal_acc(output, train_labels)

tensor(10)

In [12]:
# Set up cross-entropy loss
loss_func = nn.CrossEntropyLoss()

In [13]:
# Create a instance of Model
model = Model()

# Set up Adam optimizer, with 1e-3 learning rate and betas=(0.9, 0.999).
optimizer = optim.Adam(model.parameters(), lr=1e-3, betas = (0.9, 0.999))

In [14]:
def cal_acc(outputs, labels): 
    total = 0
    correct = 0
    _,predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    acc = torch.div(correct*100, total, rounding_mode='trunc')
    return acc

In [15]:
best_ind = -1
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
    
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
    
        #writer.add_scalar("Train Loss vs Epoches", loss, epoch)
        #writer.add_scalar("Train Accuracy(%) vs Epoches", cal_acc(outputs, labels), epoch)

    
        #print statistics
        running_loss += loss.item()
        
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
        
    train_loss = running_loss/len(train_loader)
    writer.add_scalar("Train Loss vs Epochs", train_loss, epoch)    
    
    acc = torch.div(correct*100, total, rounding_mode='trunc')
    writer.add_scalar("Train Accuracy(%) vs Epochs", acc, epoch)
    
    PATH = './checkpoints/cifar_net{:02d}.pth'.format(epoch)
    torch.save(model.state_dict(), PATH)
    
    #validation
    validation_loss = 0.0
    correct = 0
    total = 0
    
    print("starting validation for epoch {}".format(epoch+1))
    with torch.no_grad():
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            outputs = model(inputs)
            #TODO
            loss = loss_func(outputs, labels)
           
            validation_loss += loss.item()
            _,predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()

        validation_loss /= len(val_loader)
        writer.add_scalar("Validation Loss vs Epochs", validation_loss, epoch)
        acc = torch.div(correct*100, total, rounding_mode='trunc')
        writer.add_scalar("Validation Accuracy(%) vs Epochs", acc, epoch)
        
        print("validation loss for epoch {:2d}: {:5f}".format(epoch+1, validation_loss))
        print("validation acc for epoch {:2d}: {:5f}".format(epoch+1, acc))
        
print('Finished Training')
    
    
    

starting validation for epoch 1
validation loss for epoch  1: 0.259056
validation acc for epoch  1: 91.000000
starting validation for epoch 2
validation loss for epoch  2: 0.179389
validation acc for epoch  2: 94.000000
starting validation for epoch 3
validation loss for epoch  3: 0.149584
validation acc for epoch  3: 95.000000
starting validation for epoch 4
validation loss for epoch  4: 0.139131
validation acc for epoch  4: 95.000000
starting validation for epoch 5
validation loss for epoch  5: 0.119837
validation acc for epoch  5: 96.000000
starting validation for epoch 6
validation loss for epoch  6: 0.100235
validation acc for epoch  6: 97.000000
starting validation for epoch 7
validation loss for epoch  7: 0.118195
validation acc for epoch  7: 97.000000
starting validation for epoch 8
validation loss for epoch  8: 0.100965
validation acc for epoch  8: 97.000000
starting validation for epoch 9
validation loss for epoch  9: 0.083324
validation acc for epoch  9: 97.000000
starting v

In [15]:
best_ind

-1

In [16]:
len(val_loader)

16

In [28]:
model = Model()
#9th epoch
model.load_state_dict(torch.load('./checkpoints/cifar_net{:02d}.pth'.format(12)))

# Test the model on test set
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        #
        #
acc = torch.div(correct*100, total) #100 * correct // total #trunc floor
print(f"Accuracy of the network on the test images: {acc} %")


Accuracy of the network on the test images: 95.44999694824219 %
